In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

DATA_DIR = 'asteroid_images_cropped'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
    print(f"Directory '{DATA_DIR}' created.")
else:
    print(f"Directory '{DATA_DIR}' already exists.")

Directory 'asteroid_images_cropped' already exists.


In [5]:
import os

DATA_DIR = 'asteroid_images_cropped'
classes = ['S_type', 'C_type', 'M_type']

for class_name in classes:
    class_dir = os.path.join(DATA_DIR, class_name)
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)
        print(f"Directory '{class_dir}' created.")
    else:
        print(f"Directory '{class_dir}' already exists.")

Directory 'asteroid_images_cropped/S_type' created.
Directory 'asteroid_images_cropped/C_type' created.
Directory 'asteroid_images_cropped/M_type' created.


In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

# ------------------------
# Paths and parameters
# ------------------------
DATA_DIR = 'asteroid_images_cropped'  # Folder should have subfolders (one per class)
IMG_SIZE = (512, 512)
BATCH_SIZE = 16
EPOCHS = 100

# ------------------------
# Data generators
# ------------------------
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Number of classes (automatically detected from folders)
num_classes = len(train_generator.class_indices)
print("Classes found:", train_generator.class_indices)

# ------------------------
# Model definition
# ------------------------
model = Sequential([
    tf.keras.Input(shape=(*IMG_SIZE, 3)),     # ✅ Proper input layer
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')   # ✅ Matches detected classes
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# ------------------------
# Training
# ------------------------
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

# ------------------------
# Save model
# ------------------------
model.save('asteroid_classifier_model.h5')

# ------------------------
# Prediction function
# ------------------------
def predict_asteroid(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0) / 255.0
    preds = model.predict(x)
    classes = list(train_generator.class_indices.keys())
    predicted_class = classes[np.argmax(preds)]
    confidence = np.max(preds)
    return predicted_class, confidence

# Example usage:
# result, conf = predict_asteroid('test_image.jpg')
# print(f'Predicted class: {result} (confidence: {conf:.2f})')

Found 23 images belonging to 6 classes.
Found 5 images belonging to 6 classes.
Classes found: {'C-Type': 0, 'C_type': 1, 'M-Type': 2, 'M_type': 3, 'S-Type': 4, 'S_type': 5}


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 9s/step - accuracy: 0.1636 - loss: 14.0435 - val_accuracy: 0.8000 - val_loss: 8.0425
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 9s/step - accuracy: 0.4907 - loss: 8.5776 - val_accuracy: 0.8000 - val_loss: 6.6591
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 6s/step - accuracy: 0.6141 - loss: 5.6759 - val_accuracy: 0.4000 - val_loss: 4.2678
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 13s/step - accuracy: 0.5643 - loss: 5.2214 - val_accuracy: 0.6000 - val_loss: 1.7749
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step - accuracy: 0.5725 - loss: 2.7040 - val_accuracy: 0.6000 - val_loss: 0.6615
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.6431 - loss: 2.5650 - val_accuracy: 0.6000 - val_loss: 0.5244
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.9004 - loss: 0.4338 - val_accuracy: 1.0000 - val_loss: 0.4811
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 5s/step - accuracy: 1.0000 - loss: 0.4040 - val_accuracy: 1.0000 - val_loss: 0.249

In [17]:
result = predict_asteroid('/content/asteroid_images_cropped/C-Type/test_image.jpg')
print('Predicted class:', result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Predicted class: ('M-Type', np.float32(0.99995995))


In [1]:
import cv2
import os
from pathlib import Path

# ---- Cropping function ----
def crop_black_borders(input_path, output_path, min_contour_area=500):
    img = cv2.imread(str(input_path))
    if img is None:
        print(f"⚠️ Could not read {input_path}")
        return False

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print(f"⚠️ No contour found in {input_path}")
        return False

    c = max(contours, key=cv2.contourArea)
    if cv2.contourArea(c) < min_contour_area:
        print(f"⚠️ Contour too small in {input_path}")
        return False

    x, y, w, h = cv2.boundingRect(c)
    cropped = img[y:y+h, x:x+w]

    # Ensure output folder exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(str(output_path), cropped)
    return True

# ---- Apply cropping recursively ----
def preprocess_dataset(input_dir, output_dir):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)

    for class_dir in input_dir.iterdir():
        if class_dir.is_dir():
            for img_path in class_dir.glob("*.*"):
                out_path = output_dir / class_dir.name / img_path.name
                crop_black_borders(img_path, out_path)


In [12]:
INPUT_DIR = 'asteroid_images'
OUTPUT_DIR = 'asteroid_images_cropped'

preprocess_dataset(INPUT_DIR, OUTPUT_DIR)
print("✅ Cropping done, dataset ready in:", OUTPUT_DIR)


✅ Cropping done, dataset ready in: asteroid_images_cropped


In [ ]:
DATA_DIR = "asteroid_images_cropped"
